<a href="https://colab.research.google.com/github/kurtismok/automatic-tiktok-quality-analysis-tool/blob/main/Automatic_TikTok_Video_Quality_Analysis_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yt-dlp
!pip install git+https://github.com/openai/whisper.git
!apt-get update
!apt-get install -y ffmpeg
!pip install ffmpeg-python
!pip install google-generativeai
!pip install -U google-generativeai
!pip install opencv-python-headless
!pip install google-cloud-aiplatform

!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install opencv-python pytesseract


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.6 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5ymstq8i
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-5ymstq8i
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━

In [ ]:
from yt_dlp import YoutubeDL
import whisper
import ffmpeg
import google.generativeai as genai
import os
from vertexai.generative_models import GenerativeModel, Part
from google.cloud import storage

import vertexai
from vertexai.generative_models import GenerativeModel


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [52]:
PROJECT_ID = "my-video-analyzer"
LOCATION = "us-central1"
BUCKET_NAME = "my-video-analyzer-bucket"

vertexai.init(project = PROJECT_ID, location = LOCATION)

def upload_to_gcs(file, bucket_name):
    """Uploads a file to the specified GCS bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    get_base_filename = os.path.basename(file)
    blob = bucket.blob(get_base_filename)

    blob.upload_from_filename(file)
    print(f"File {file} uploaded to gs://{bucket_name}/{get_base_filename}")
    return f"gs://{bucket_name}/{get_base_filename}"

def download_video(url):
    video_id = url.rstrip('/').split('/')[-1]
    output_path = f"{video_id}.mp4"
    ydl_opts = {'format': 'mp4', 'outtmpl': output_path}
    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return output_path

def transcribe_audio(video_path, model_name='base'):
    model = whisper.load_model(model_name)
    audio_path = "temp_audio.wav"
    try:
        (
            ffmpeg.input(video_path)
            .output(audio_path, ac=1, ar='16000')
            .overwrite_output()
            .run(quiet=True, capture_stderr=True)
        )
    except ffmpeg.Error as e:
        print("FFmpeg error:", e.stderr.decode('utf8'))
        raise
    result = model.transcribe(audio_path, fp16=False)
    os.remove(audio_path)
    return result['text']


In [54]:
gemini_model = GenerativeModel('gemini-2.0-flash-001')
def call_gemini_with_video(ref_video_uri, user_video_uri, ref_transcript, user_transcript):
    # Create Part objects from the GCS URIs
    ref_video_part = Part.from_uri(ref_video_uri, mime_type="video/mp4")
    user_video_part = Part.from_uri(user_video_uri, mime_type="video/mp4")

    # This prompt tells Gemini to WATCH the videos and use the rubric.
    prompt = f"""
    Given the brand reference video description: "{ref_transcript}"
    And the uploaded video description: "{user_transcript}"
    Rate the uploaded video's quality from 0 to 100 based on tone, clarity, and brand consistency
    using the reference video provided as the ideal tone/manner.
    Show the evaluated scoire and then give feedback as a list of points (e.g., “Lighting too dark”, “Brand name not mentioned”)
    Then provide a recommendation to either pass, re-shoot, or manual review the video.
    """

    response = gemini_model.generate_content([
        ref_video_part,
        user_video_part,
        prompt
    ])
    return response.text



In [42]:
def main(reference_video_url, user_video_url):
    print("Downloading Videos")
    reference_video_path = download_video(reference_video_url)
    user_video_path = download_video(user_video_url)

    print("\nUploading Videos to Google Cloud Storage")
    ref_gcs_uri = upload_to_gcs(reference_video_path, BUCKET_NAME)
    user_gcs_uri = upload_to_gcs(user_video_path, BUCKET_NAME)

    print("\Transcribing Audio")
    ref_transcript = transcribe_audio(reference_video_path)
    user_transcript = transcribe_audio(user_video_path)

    print("\nCalling Gemini for Analysis")
    gemini_result = call_gemini_with_video(ref_gcs_uri, user_gcs_uri, ref_transcript, user_transcript)

    print("\n=== Quality Evaluation Result ===")
    print(gemini_result)

In [64]:
# Input TikTok URLs here:
reference_url = "https://www.tiktok.com/@age20s_global/video/7495561195261365521"
user_url = "https://www.tiktok.com/@age20s_global/video/7472550447769799952"

main(reference_url, user_url)

[TikTok] Extracting URL: https://www.tiktok.com/@age20s_global/video/7495561195261365521
[TikTok] 7495561195261365521: Downloading webpage
[info] 7495561195261365521: Downloading 1 format(s): h264_540p_1081337-1
[download] 7495561195261365521.mp4 has already been downloaded
[download] 100% of    3.98MiB
[TikTok] Extracting URL: https://www.tiktok.com/@age20s_global/video/7472550447769799952
[TikTok] 7472550447769799952: Downloading webpage
[info] 7472550447769799952: Downloading 1 format(s): bytevc1_1080p_800237-1
[download] 7472550447769799952.mp4 has already been downloaded
[download] 100% of    1.03MiB

Uploading Videos to Google Cloud Storage
File 7495561195261365521.mp4 uploaded to gs://my-video-analyzer-bucket/7495561195261365521.mp4
File 7472550447769799952.mp4 uploaded to gs://my-video-analyzer-bucket/7472550447769799952.mp4
\Transcribing Audio

Calling Gemini for Analysis

=== Quality Evaluation Result ===
Okay, I can evaluate the uploaded video and provide feedback based on t